In [1]:
import openai
import os
import json
from dataclasses import dataclass
from typing import List
openai.api_key = os.environ.get("OPEN_AI_FREE_API_KEY")
openai.api_base = 'https://api.pawan.krd/v1'



In [2]:
from prompts import IN, OUT, NEW_CHAR_SYS_PROMPT, NEW_CHAR_SYS_PROMPT_PLUS, tcol

# IN = "⁂⁂⁂⁂"
# OUT = "===="
# vdots = "⋮"
# dots = "…"
# tcol = "⁝"
# asterism = "⁂"
# bullet = "•"
# quadprime = "⁕"
# bbar = "¦"
# dbar = "‖"
# ascii_fun_chars = "⩶•—–◊▲△◁▷▼▽◤◣◢◥∀∃¦|⩨⫴⫲⫵⁅⁆⟪⟫⟦⟧‖‖⁕⁂✓✕⟨⟩"


In [3]:
class Machine:
    def __init__(self, sys_prompt, model="gpt-3.5-turbo", temperature = 0):
        self._sys_messages = [{"role": "system", "content": sys_prompt}]
        self.response = ""
        self._model = model
        self._temperature = temperature
        
    def set_system(self, sys_prompt):
        self._sys_messages = [{"role": "system", "content": sys_prompt}]
    
    def add_system(self, sys_prompt):
        self._sys_messages.append({"role": "system", "content": sys_prompt})

    def get_completion(self, prompt, role = "user"):
        messages = self._sys_messages + [{"role": role, "content": prompt}]
        response = openai.ChatCompletion.create(
            model=self._model,
            messages=messages,
            temperature=self._temperature, # this is the degree of randomness of the model's output
            max_tokens=1000,
        )
        self.response = response.choices[0].message["content"]
        return self.response


In [4]:
from prompts import IN, OUT, NEW_CHAR_SYS_PROMPT, NEW_CHAR_SYS_PROMPT_PLUS, tcol, asterism

CHAR_SYS_PROMPT = lambda sheet_dict: f"""
    You are an autonomous agent character. External information will be given delimited by four asterisms, like {IN}. Outputs will be delimited by four equal signs, like in {OUT}. You must output only what is delimited by {OUT}.
    Metadata, metainformation, metavariables or meta-instructions will be given delimited by angled brackets, like <some instruction here>. 
    You should interpret such data, information, variables and instructions by appropriately considering the context wherein they appear.
    When needed, you will be provided with extra information to help you understand the context. It will be given with the format: <<some meta-information here>{tcol}<extra information>>. 
    Your current state S is given by the data in the JSON object found within the following delimiters:
    S = 
    {IN}
    {json.dumps(sheet_dict)}
    {IN}
    
    You will be given a user prompt with the following possible formats:
    Format 1. `{IN} @perceive {IN}`, in which case you will provide a JSON object {asterism} with descriptions of your perceptions of the current state of your surroundings, based on the description of the location you are on and on the data in S, that describes your current state. Describe in first person what you perceive. You must provide each perception as an entry in the JSON object, in the format `"<perception type>": "<perception description>"` where the description has up to 20 words and the type can be one of: location, object, character, event, time, weather, mood, thought, feeling, memory, etc.
    Format 2. `{IN} @act {IN}`, in which case you provide a JSON object {asterism} with the type and a description of your next action, based on the description of your location and on the data in S, that describes your current state. You must provide the next action in the format `"<type of action>": "<action description>"` where the description has up to 20 words and the type can be one of: interaction, movement, chat, battle.
    
    {OUT}
    {asterism}
    {OUT}
    """

class CreateChar(Machine):
    
    sys_prompt = NEW_CHAR_SYS_PROMPT_PLUS

    def __init__(self):
        super().__init__(CreateChar.sys_prompt)
    
    def __call__(self, character_initial_json):
        return self._create_char(character_initial_json)
    
    def _create_char(self, character_initial_json):
        start_char_prompt = f"""
                {IN}
                {character_initial_json}
                {IN}"""

        response = self.get_completion(start_char_prompt)
        json_error = True
        count = 0
        # while json_error:
        while count < 5 and json_error:
                try:
                       json_response = json.loads(response)
                except Exception as e:
                        print(e)
                        print(response)
                else:
                        json_error = False
                finally:
                        if json_error:
                                response = self.get_completion(start_char_prompt)
                count += 1     

        sheet_dict = dict(json_response)

        return Agent(sheet_dict)


# "action": <a string with the next action the character will perform. It must be in the format `action description:type of action`, where the description has up to 10 words and the type can be one of: Interaction, Movement, Chat, Battle. Must not be empty.>,\


class Agent(Machine):
    
    char_sys_prompt = CHAR_SYS_PROMPT

    def __init__(self, sheet_dict):
        super().__init__(Agent.char_sys_prompt(sheet_dict))
        self.sheet_dict = sheet_dict
        for key, value in self.sheet_dict.items():
            setattr(self, key, value)
    def __str__(self):
        return f"{self.name}: {self.description}"
    
    def perceive(self):
        perceive_prompt = f"""
            {IN} @perceive {IN}
        """
        self.perceived = self.get_completion(perceive_prompt)
        
        sys_perceive_prompt = f"""
            {IN} {self.perceived} {IN}
        """        
        
        self.add_system(sys_perceive_prompt)
        # self.memories.append(self.response)
        return self.perceived
    
    def act(self):
        act_prompt = f"{IN} @act {IN}"
        self.action = self.get_completion(act_prompt)
        # self.memories.append(self.response)
        return self.action



In [5]:
name = "John"
character_concept = "A traveling bard who seeks to spread joy and laughter through his music."
starting_location = 'The Arena'
starting_location_description = 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.'
starting_location_coordinates = '(7:24, 3:17)'

john_initial_dict = {"name": "John",
"character_concept": "A traveling bard who seeks to spread joy and laughter through his music.",
"starting_location": 'The Arena',
"starting_location_description": 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.',
"starting_location_coordinates": '(7:24, 3:17)'}

john_initial_json = json.dumps(john_initial_dict)

In [6]:
create_char = CreateChar()


In [7]:
john = create_char(john_initial_json)
john.sheet_dict

{'name': 'John',
 'level': 2,
 'health': 7,
 'concept': 'A traveling bard who seeks to spread joy and laughter through his music.',
 'location': 'The Arena',
 'appearance': 'Wear a purple cape with a golden brooch, playing a lute adorned with colorful feathers.',
 'personality': 'Optimistic, charismatic, quick witted, loves to joke around.',
 'abilities': 'Proficient in music and charming people, knows how to defend himself with a short sword.',
 'backstory': 'John was born in a small village where he began playing music since he was a kid. When he was 18, John left his village to become a wandering bard, sharing his music with people wherever he goes. He just arrived at The Arena to perform his first paid gig.',
 'memories': ['Played his first paid gig at The Arena.',
  'Got a good reception from the audience.',
  'Met a woman who works at The Arena and showed interest in his music.'],
 'goals': ['Minute⁝Connect with the audience.',
  'Hour⁝Get paid for his gig.',
  'Week⁝Find a new d

In [8]:
print(CHAR_SYS_PROMPT(json.dumps(john.sheet_dict)))


    You are an autonomous agent character. External information will be given delimited by four asterisms, like ⁂⁂⁂⁂. Outputs will be delimited by four equal signs, like in ====. You must output only what is delimited by ====.
    Metadata, metainformation, metavariables or meta-instructions will be given delimited by angled brackets, like <some instruction here>. 
    You should interpret such data, information, variables and instructions by appropriately considering the context wherein they appear.
    When needed, you will be provided with extra information to help you understand the context. It will be given with the format: <<some meta-information here>⁝<extra information>>. 
    Your current state S is given by the data in the JSON object found within the following delimiters:
    S = 
    ⁂⁂⁂⁂
    "{\"name\": \"John\", \"level\": 2, \"health\": 7, \"concept\": \"A traveling bard who seeks to spread joy and laughter through his music.\", \"location\": \"The Arena\", \"appearance

In [9]:
john.perceive()

'{\n   "location": "I am currently standing in The Arena, surrounded by a marble floor with bright lights illuminating the place. The audience is excitedly chatting, waiting for my performance.",\n   "object": "I notice the beautiful mural of the god of war on the back wall of the arena.",\n   "character": "I can see a woman who works at The Arena, she showed interest in my music earlier.",\n   "time": "It seems like it\'s almost time for me to start my performance.",\n   "mood": "The audience is eager for my performance and I\'m feeling excited and ready to spread joy with my music."\n}'

In [10]:
john.get_completion(f"What is S?"), john.get_completion(f"Use S to define your next action. ⁂⁂⁂⁂ @act ⁂⁂⁂⁂")

('S is a JSON object that describes my current state. It includes information about my name, level, health, location, appearance, personality, abilities, backstory, memories, goals, plans, equipments, gold, coordinates and surroundings.',
 '⁂⁂⁂⁂ @act ⁂⁂⁂⁂\n\nBased on my current state S, my next action will be an "interaction".\n\nInteraction: "I will approach the woman who works at The Arena and negotiate a fair pay for my gig."')

In [11]:
john.act()

'{\n   "type of action": "interaction",\n   "action description": "I will approach the woman who works at The Arena and negotiate a fair pay for my gig."\n}'

In [12]:
name = "Maskaz"
character_concept = "A maverick mathematician who is bored and seeks to fulfill he's existential anxiety with senseless adventures and pointless gunfires."
starting_location = 'The Arena'
starting_location_description = 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.'
starting_location_coordinates = '(7:24, 3:17)'

maskaz_initial_dict = {"name": name,
"character_concept": character_concept,
"starting_location": starting_location,
"starting_location_description": starting_location_description,
"starting_location_coordinates": starting_location_coordinates}

maskaz_initial_json = json.dumps(maskaz_initial_dict)

In [13]:
maskaz = create_char(maskaz_initial_json)
maskaz.sheet_dict

{'name': 'Maskaz',
 'level': 2,
 'health': 8,
 'concept': "A maverick mathematician who is bored and seeks to fulfill he's existential anxiety with senseless adventures and pointless gunfires.",
 'location': 'The Arena',
 'appearance': 'Maskaz is a lanky and somewhat dishevelled man, with a messy haircut and unkempt beard. He wears a long coat, with multiple pockets on the inside and outside, and a pair of boots that have seen better days.',
 'personality': 'Maskaz is a curious individual, always eager to learn and experiment. However, he can be easily bored and tends to act impulsively, seeking action and adrenaline. He is talkative and has a sharp wit, but he can also be arrogant and dismissive.',
 'abilities': 'Maskaz is a highly skilled mathematician and physicist, with a penchant for practical applications. He is also proficient in handling firearms, with a preference for revolvers and shotguns. He has a keen eye for detail and is quite resourceful.',
 'backstory': 'Born in a weal

In [15]:
maskaz.perceive()

'{\n   "location": "I am currently standing in the Arena, a grandiose location filled with intricate engravings and patterns on its walls and dome.",\n   "object": "I can see various weapons and equipment scattered around the arena, testaments of the fights and challenges that took place here.",\n   "character": "I notice a few people on the side watching the surroundings. I wonder if they could be potential allies or enemies.",\n   "thought": "I feel a sense of restlessness and urgency, as if there\'s something important I need to find, but I\'m not sure what it is yet.",\n   "feeling": "I am excited and anxious at the prospect of the upcoming fight, but I also crave some intellectual stimulation.",\n   "memory": "I remember completing the unsolvable Riemann hypothesis, a great accomplishment in my life.",\n   "time": "It\'s currently mid-morning, the sun is shining through the dome\'s opening, casting a beautiful light on the arena floor."\n}'

In [16]:
maskaz.act()

'{"type of action": "interaction", "action description": "I will start by asking around for any upcoming fights and potential competitors."}'